# Data requirements and preprocessing

## We require three types of data: Mutation, Copy Number, and Response (the label).

Here, as an example, we preprocess data from Nguyen et al., Cell 2022, which contains a comprehensive dataset; however, we only require the following three data types:

① data_mutation.txt
② data_cna.txt
③ data_clinical_sample.txt


Data Link：https://www.cbioportal.org/study/summary?id=msk_met_2021

### 1. Mutation ———— data_mutation.txt

In [1]:
import pandas as pd
from os.path import join

data_mutations_filename = 'data_mutations.txt'

mutations = pd.read_csv(data_mutations_filename, sep='\t', low_memory=False, skiprows=0)

id_col = 'Tumor_Sample_Barcode'

mutations['Variant_Classification'].value_counts()

#exclude = ['Silent', 'Intron', "3\'UTR", "5\'UTR", 'RNA', 'lincRNA']
exclude = ['Silent', 'Intron', "5\'UTR"]

mutations = mutations[~mutations['Variant_Classification'].isin(exclude)].copy()

mutations_table = pd.pivot_table(data=mutations, index=id_col, columns='Hugo_Symbol', values='Variant_Classification',aggfunc='count')

mutations_table = mutations_table.fillna(0)

total_numb_mutations = mutations_table.sum().sum()
number_samples = mutations_table.shape[0]

print('number of mutations:', total_numb_mutations, 'average mutation/sample:', total_numb_mutations / (number_samples + 0.0))

mutations_table.to_csv('msk_2021_mut.csv')

number of mutations: 230242.0 average mutation/sample: 9.301579606512341


### 2. Copy Number ———— data_cna.txt

In [2]:
data_copy_number_filename = 'data_cna.txt'
cna = pd.read_csv(data_copy_number_filename, sep='\t', low_memory=False, skiprows=0, index_col=0)

cna = cna.T
cna = cna.fillna(0.)

cna.to_csv('msk_2021_cna.csv')

### 3. Response ———— data_clinical_sample.txt

In [3]:
data_response_filename = 'data_clinical_sample.txt'

data_response = pd.read_csv(data_response_filename, sep='\t', low_memory=False, skiprows=4)

response = data_response[['SAMPLE_ID', 'SAMPLE_TYPE']]
response = response.rename(columns={"SAMPLE_ID": "id", "SAMPLE_TYPE": "response"})

response['response'] = response['response'].replace('Metastasis', 1)
response['response'] = response['response'].replace('Primary', 0)
response = response.drop_duplicates()

response.to_csv('msk_2021_response.csv')